In [1]:
import os
import sys
import SimpleITK as sitk
import numpy as np
from PIL import Image
import cv2
from matplotlib import pyplot as plot
import pandas as pd
import xlrd

In [19]:
model_name = "transbts"
result_dir = f'/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/result/狭窄率图表_0916/{model_name}/prediction'
file_list = os.listdir(result_dir)
try:
    file_list.remove(".DS_Store")
except ValueError:
    pass
try:
    file_list.remove("._.DS_Store")
except ValueError:
    pass

save_path = f"/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/result/狭窄率图表_0916/results/{model_name}"

if not os.path.exists(save_path):
    os.makedirs(save_path)

for path in file_list:
    data = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(result_dir, path)))
    name = path[:-18]
    out = sitk.GetImageFromArray(data)
    sitk.WriteImage(out, f'{save_path}/{name}.nii.gz')

In [8]:
# 计算狭窄率，对外膜的mask进行腐蚀操作，去除正常血管壁的厚度
model_name = "label"
result_dir = f'/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/result/狭窄率图表_0916/results/{model_name}'
file_list = os.listdir(result_dir)
try:
    file_list.remove(".DS_Store")
except ValueError:
    pass
try:
    file_list.remove("._.DS_Store")
except ValueError:
    pass

pn_all = {}
pne_all = {}
for idx, file_name in enumerate(file_list):
    if file_name.endswith(".nii.gz"):
        data = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(result_dir, file_name)))
    elif file_name.endswith(".npy"):
        data = np.load(os.path.join(result_dir, file_name), allow_pickle=True)
    else:
        print(f"data format {os.path.splitext(file_name)[-1]} is error")
        sys.exit()

    pn = []
    pne = []
    for k, img in enumerate(data):
        # pred 腐蚀
        out_mask = np.zeros_like(img)
        out_mask[img == 1] = 1  # 环形mask
        in_mask = np.zeros_like(img)
        in_mask[img == 2] = 1  # 内膜mask
        both_mask = out_mask + in_mask  # 外膜mask
        both_mask_erode = cv2.erode(both_mask,
                                    kernel=(3, 3),
                                    iterations=3)

        pred_narrow = 1 - (np.sum(in_mask) / np.sum(both_mask))
        pred_erode_narrow = 1 - (np.sum(in_mask) / np.sum(both_mask_erode))

        pn.append(pred_narrow)
        pne.append(pred_erode_narrow)

        if False:
            print(f'pred_narrow:{pred_narrow:.2f} | pred_narrow:{pred_erode_narrow:.2f}')

    pn_all[file_name] = pd.Series(pn)
    pne_all[file_name] = pd.Series(pne)

if False:
    with pd.ExcelWriter(f'{model_name}.xlsx') as writer:
        df_pn_all = pd.DataFrame(data=pn_all)
        df_pn_all.to_excel(writer,
                           sheet_name="diameter stenosis",
                           index=True,
                           header=True,
                           startrow=0,
                           startcol=0)

    with pd.ExcelWriter(f'{model_name}_erode.xlsx') as writer:
        df_pne_all = pd.DataFrame(data=pne_all)
        df_pne_all.to_excel(writer,
                            sheet_name="diameter stenosis",
                            index=True,
                            header=True,
                            startrow=0,
                            startcol=0)

In [11]:
print(sorted(list(pn_all.keys())))

['02220820210305_XJ_20210305082931543.nii.gz', '06570820220302_CA MASHAOYONG_20220302091057736.nii.gz', '06570820220302_CA MASHAOYONG_20220302091109144.nii.gz', '202203081456170036VAS.nii.gz', '202203140858390011VAS.nii.gz', '202203141429300009VAS.nii.gz', '202203211455210024VAS.nii.gz', '202203241105420022VAS.nii.gz', '202203241107360023VAS.nii.gz', '202203251425020010VAS.nii.gz', '202204020917510011VAS.nii.gz', '202204070858070033VAS.nii.gz', '202204110951560016VAS.nii.gz', '24060820210210_SMY_20210210081527458.nii.gz', '29081020210319_YZX_20210319101543139.nii.gz', '39561320210331_DJF_20210331140409947.nii.gz', '43291020210205_CQH_20210205103732049.nii.gz', 'IM_0010.nii.gz', 'IM_0033.nii.gz', 'IM_0039.nii.gz', 'IM_0044.nii.gz', 'IM_0049.nii.gz', 'IM_0061.nii.gz', 'IM_0079.nii.gz', 'IM_0084.nii.gz', '_20210128075939_0818310.nii.gz', '_20210224093839_0943280.nii.gz']


In [9]:
# 计算狭窄率，对外膜的mask进行腐蚀操作，去除正常血管壁的厚度
model_name = "3dunet"
result_dir = f'/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/result/狭窄率图表_0916/results/{model_name}'
file_list = os.listdir(result_dir)
try:
    file_list.remove(".DS_Store")
except ValueError:
    pass
try:
    file_list.remove("._.DS_Store")
except ValueError:
    pass

pn_all_1 = {}
pne_all_1 = {}
for idx, file_name in enumerate(file_list):
    if file_name.endswith(".nii.gz"):
        data = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(result_dir, file_name)))
    elif file_name.endswith(".npy"):
        data = np.load(os.path.join(result_dir, file_name), allow_pickle=True)
    else:
        print(f"data format {os.path.splitext(file_name)[-1]} is error")
        sys.exit()

    pn = []
    pne = []
    for k, img in enumerate(data):
        # pred 腐蚀
        out_mask = np.zeros_like(img)
        out_mask[img == 1] = 1  # 环形mask
        in_mask = np.zeros_like(img)
        in_mask[img == 2] = 1  # 内膜mask
        both_mask = out_mask + in_mask  # 外膜mask
        both_mask_erode = cv2.erode(both_mask,
                                    kernel=(3, 3),
                                    iterations=3)

        pred_narrow = 1 - (np.sum(in_mask) / np.sum(both_mask))
        pred_erode_narrow = 1 - (np.sum(in_mask) / np.sum(both_mask_erode))

        pn.append(pred_narrow)
        pne.append(pred_erode_narrow)

        if False:
            print(f'pred_narrow:{pred_narrow:.2f} | pred_narrow:{pred_erode_narrow:.2f}')

    pn_all_1[file_name] = pd.Series(pn)
    pne_all_1[file_name] = pd.Series(pne)

if False:
    with pd.ExcelWriter(f'{model_name}.xlsx') as writer:
        df_pn_all = pd.DataFrame(data=pn_all_1)
        df_pn_all.to_excel(writer,
                           sheet_name="diameter stenosis",
                           index=True,
                           header=True,
                           startrow=0,
                           startcol=0)

    with pd.ExcelWriter(f'{model_name}_erode.xlsx') as writer:
        df_pne_all = pd.DataFrame(data=pne_all_1)
        df_pne_all.to_excel(writer,
                            sheet_name="diameter stenosis",
                            index=True,
                            header=True,
                            startrow=0,
                            startcol=0)

In [12]:
print(sorted(list(pn_all_1.keys())))

['02220820210305_XJ_20210305082931543.nii.gz', '06570820220302_CA MASHAOYONG_20220302091057736.nii.gz', '06570820220302_CA MASHAOYONG_20220302091109144.nii.gz', '202203081456170036VAS.nii.gz', '202203140858390011VAS.nii.gz', '202203141429300009VAS.nii.gz', '202203211455210024VAS.nii.gz', '202203241105420022VAS.nii.gz', '202203241107360023VAS.nii.gz', '202203251425020010VAS.nii.gz', '202204020917510011VAS.nii.gz', '202204070858070033VAS.nii.gz', '202204110951560016VAS.nii.gz', '24060820210210_SMY_20210210081527458.nii.gz', '29081020210319_YZX_20210319101543139.nii.gz', '39561320210331_DJF_20210331140409947.nii.gz', '43291020210205_CQH_20210205103732049.nii.gz', 'IM_0010.nii.gz', 'IM_0033.nii.gz', 'IM_0039.nii.gz', 'IM_0044.nii.gz', 'IM_0049.nii.gz', 'IM_0061.nii.gz', 'IM_0079.nii.gz', 'IM_0084.nii.gz', '_20210128075939_0818310.nii.gz', '_20210224093839_0943280.nii.gz']


In [13]:
# 计算狭窄率，对外膜的mask进行腐蚀操作，去除正常血管壁的厚度
model_name = "rvm"
result_dir = f'/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/result/狭窄率图表_0916/results/{model_name}'
file_list = os.listdir(result_dir)
try:
    file_list.remove(".DS_Store")
except ValueError:
    pass
try:
    file_list.remove("._.DS_Store")
except ValueError:
    pass

pn_all_2 = {}
pne_all_2 = {}
for idx, file_name in enumerate(file_list):
    if file_name.endswith(".nii.gz"):
        data = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(result_dir, file_name)))
    elif file_name.endswith(".npy"):
        data = np.load(os.path.join(result_dir, file_name), allow_pickle=True)
    else:
        print(f"data format {os.path.splitext(file_name)[-1]} is error")
        sys.exit()

    pn = []
    pne = []
    for k, img in enumerate(data):
        # pred 腐蚀
        out_mask = np.zeros_like(img)
        out_mask[img == 1] = 1  # 环形mask
        in_mask = np.zeros_like(img)
        in_mask[img == 2] = 1  # 内膜mask
        both_mask = out_mask + in_mask  # 外膜mask
        both_mask_erode = cv2.erode(both_mask,
                                    kernel=(3, 3),
                                    iterations=3)

        pred_narrow = 1 - (np.sum(in_mask) / np.sum(both_mask))
        pred_erode_narrow = 1 - (np.sum(in_mask) / np.sum(both_mask_erode))

        pn.append(pred_narrow)
        pne.append(pred_erode_narrow)

        if False:
            print(f'pred_narrow:{pred_narrow:.2f} | pred_narrow:{pred_erode_narrow:.2f}')

    pn_all_2[file_name] = pd.Series(pn)
    pne_all_2[file_name] = pd.Series(pne)

if False:
    with pd.ExcelWriter(f'{model_name}.xlsx') as writer:
        df_pn_all = pd.DataFrame(data=pn_all_2)
        df_pn_all.to_excel(writer,
                           sheet_name="diameter stenosis",
                           index=True,
                           header=True,
                           startrow=0,
                           startcol=0)

    with pd.ExcelWriter(f'{model_name}_erode.xlsx') as writer:
        df_pne_all = pd.DataFrame(data=pne_all_2)
        df_pne_all.to_excel(writer,
                            sheet_name="diameter stenosis",
                            index=True,
                            header=True,
                            startrow=0,
                            startcol=0)

In [14]:
print(sorted(list(pn_all_2.keys())))

['02220820210305_XJ_20210305082931543.nii.gz', '06570820220302_CA MASHAOYONG_20220302091057736.nii.gz', '06570820220302_CA MASHAOYONG_20220302091109144.nii.gz', '202203081456170036VAS.nii.gz', '202203140858390011VAS.nii.gz', '202203141429300009VAS.nii.gz', '202203211455210024VAS.nii.gz', '202203241105420022VAS.nii.gz', '202203241107360023VAS.nii.gz', '202203251425020010VAS.nii.gz', '202204020917510011VAS.nii.gz', '202204070858070033VAS.nii.gz', '202204110951560016VAS.nii.gz', '24060820210210_SMY_20210210081527458.nii.gz', '29081020210319_YZX_20210319101543139.nii.gz', '39561320210331_DJF_20210331140409947.nii.gz', '43291020210205_CQH_20210205103732049.nii.gz', 'IM_0010.nii.gz', 'IM_0033.nii.gz', 'IM_0039.nii.gz', 'IM_0044.nii.gz', 'IM_0049.nii.gz', 'IM_0061.nii.gz', 'IM_0079.nii.gz', 'IM_0084.nii.gz', '_20210128075939_0818310.nii.gz', '_20210224093839_0943280.nii.gz']


In [15]:
# 计算狭窄率，对外膜的mask进行腐蚀操作，去除正常血管壁的厚度
model_name = "transbts"
result_dir = f'/Volumes/昊大侠/工作/实习相关/微创卜算子医疗科技有限公司/陈嘉懿组/数据/短轴动态狭窄率/result/狭窄率图表_0916/results/{model_name}'
file_list = os.listdir(result_dir)
try:
    file_list.remove(".DS_Store")
except ValueError:
    pass
try:
    file_list.remove("._.DS_Store")
except ValueError:
    pass

pn_all_3 = {}
pne_all_3 = {}
for idx, file_name in enumerate(file_list):
    if file_name.endswith(".nii.gz"):
        data = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(result_dir, file_name)))
    elif file_name.endswith(".npy"):
        data = np.load(os.path.join(result_dir, file_name), allow_pickle=True)
    else:
        print(f"data format {os.path.splitext(file_name)[-1]} is error")
        sys.exit()

    pn = []
    pne = []
    for k, img in enumerate(data):
        # pred 腐蚀
        out_mask = np.zeros_like(img)
        out_mask[img == 1] = 1  # 环形mask
        in_mask = np.zeros_like(img)
        in_mask[img == 2] = 1  # 内膜mask
        both_mask = out_mask + in_mask  # 外膜mask
        both_mask_erode = cv2.erode(both_mask,
                                    kernel=(3, 3),
                                    iterations=3)

        pred_narrow = 1 - (np.sum(in_mask) / np.sum(both_mask))
        pred_erode_narrow = 1 - (np.sum(in_mask) / np.sum(both_mask_erode))

        pn.append(pred_narrow)
        pne.append(pred_erode_narrow)

        if False:
            print(f'pred_narrow:{pred_narrow:.2f} | pred_narrow:{pred_erode_narrow:.2f}')

    pn_all_3[file_name] = pd.Series(pn)
    pne_all_3[file_name] = pd.Series(pne)

if False:
    with pd.ExcelWriter(f'{model_name}.xlsx') as writer:
        df_pn_all = pd.DataFrame(data=pn_all_3)
        df_pn_all.to_excel(writer,
                           sheet_name="diameter stenosis",
                           index=True,
                           header=True,
                           startrow=0,
                           startcol=0)

    with pd.ExcelWriter(f'{model_name}_erode.xlsx') as writer:
        df_pne_all = pd.DataFrame(data=pne_all_3)
        df_pne_all.to_excel(writer,
                            sheet_name="diameter stenosis",
                            index=True,
                            header=True,
                            startrow=0,
                            startcol=0)

In [16]:
print(sorted(list(pn_all_3.keys())))

['02220820210305_XJ_20210305082931543.nii.gz', '06570820220302_CA MASHAOYONG_20220302091057736.nii.gz', '06570820220302_CA MASHAOYONG_20220302091109144.nii.gz', '202203081456170036VAS.nii.gz', '202203140858390011VAS.nii.gz', '202203141429300009VAS.nii.gz', '202203211455210024VAS.nii.gz', '202203241105420022VAS.nii.gz', '202203241107360023VAS.nii.gz', '202203251425020010VAS.nii.gz', '202204020917510011VAS.nii.gz', '202204070858070033VAS.nii.gz', '202204110951560016VAS.nii.gz', '24060820210210_SMY_20210210081527458.nii.gz', '29081020210319_YZX_20210319101543139.nii.gz', '39561320210331_DJF_20210331140409947.nii.gz', '43291020210205_CQH_20210205103732049.nii.gz', 'IM_0010.nii.gz', 'IM_0033.nii.gz', 'IM_0039.nii.gz', 'IM_0044.nii.gz', 'IM_0049.nii.gz', 'IM_0061.nii.gz', 'IM_0079.nii.gz', 'IM_0084.nii.gz', '_20210128075939_0818310.nii.gz', '_20210224093839_0943280.nii.gz']


In [ ]:
save_dir = f"/Users/WangHao/Desktop/TODO/Data/figs"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

for idx in range(len(pn_all)):
    assert sorted(list(pn_all.keys()))[idx] == sorted(list(pn_all_1.keys()))[idx] == sorted(list(
        pn_all_2.keys()))[idx] == sorted(list(pn_all_3.keys()))[idx]

    title = list(pn_all.keys())[idx][:-7]

    fig_x = list(pn_all[sorted(list(pn_all.keys()))[idx]].index)
    fig_y = list(pn_all[sorted(list(pn_all.keys()))[idx]].values)

    fig_x_1 = list(pn_all_1[sorted(list(pn_all_1.keys()))[idx]].index)
    fig_y_1 = list(pn_all_1[sorted(list(pn_all_1.keys()))[idx]].values)

    fig_x_2 = list(pn_all_2[sorted(list(pn_all_2.keys()))[idx]].index)
    fig_y_2 = list(pn_all_2[sorted(list(pn_all_2.keys()))[idx]].values)

    fig_x_3 = list(pn_all_3[sorted(list(pn_all_3.keys()))[idx]].index)
    fig_y_3 = list(pn_all_3[sorted(list(pn_all_3.keys()))[idx]].values)

    plot.rcParams.update({"font.size": 10})
    fig = plot.figure(idx, figsize=(18, 6), dpi=300)
    ax = fig.add_subplot(111)

    ax.set(title=title,
           xlabel='frame',
           ylabel='diameter stenosis',
           ylim=[0, 1])

    ax.plot(fig_x, fig_y, 'm', label='label')

    ax.plot(fig_x_1, fig_y_1, 'g', label='3dunet')

    ax.plot(fig_x_2, fig_y_2, 'c', label='rvm')

    ax.plot(fig_x_3, fig_y_3, 'b', label='transbts')

    ax.legend()
    ax.grid(True)

    fig.savefig(f'{save_dir}/{title}.png')

In [ ]:
save_dir = f"/Users/WangHao/Desktop/TODO/Data/figs"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

for idx in range(len(pn_all)):
    assert sorted(list(pn_all.keys()))[idx] == sorted(list(pn_all_1.keys()))[idx] == sorted(list(
        pn_all_2.keys()))[idx] == sorted(list(pn_all_3.keys()))[idx]

    title = os.path.splitext(list(pn_all.keys())[idx])[0]

    fig_x = list(pn_all[sorted(list(pn_all.keys()))[idx]].index)
    fig_y = list(pn_all[sorted(list(pn_all.keys()))[idx]].values)

    fig_x_1 = list(pn_all_1[sorted(list(pn_all_1.keys()))[idx]].index)
    fig_y_1 = list(pn_all_1[sorted(list(pn_all_1.keys()))[idx]].values)

    fig_x_2 = list(pn_all_2[sorted(list(pn_all_2.keys()))[idx]].index)
    fig_y_2 = list(pn_all_2[sorted(list(pn_all_2.keys()))[idx]].values)

    fig_x_3 = list(pn_all_3[sorted(list(pn_all_3.keys()))[idx]].index)
    fig_y_3 = list(pn_all_3[sorted(list(pn_all_3.keys()))[idx]].values)

    fig = plot.figure(idx, figsize=(18, 4), dpi=300)
    plot.rcParams.update({"font.size": 8})
    ax = fig.add_subplot(141)
    ax.set(title="transbts", xlabel='frame', ylabel='diameter stenosis')
    ax.plot(fig_x, fig_y)
    ax.text(fig_y.index(max(fig_y)),
            max(fig_y), (fig_y.index(max(fig_y)), round(max(fig_y), 2)),
            color='r')
    ax.grid(True)

    ax1 = fig.add_subplot(142)
    ax1.set(title="rvm", xlabel='frame', ylabel='diameter stenosis')
    ax1.plot(fig_x_1, fig_y_1)
    ax1.text(fig_y_1.index(max(fig_y_1)),
             max(fig_y_1),
             (fig_y_1.index(max(fig_y_1)), round(max(fig_y_1), 2)),
             color='r')
    ax1.grid(True)

    ax2 = fig.add_subplot(143)
    ax2.set(title="3dunet", xlabel='frame', ylabel='diameter stenosis')
    ax2.plot(fig_x_2, fig_y_2)
    ax2.text(fig_y_2.index(max(fig_y_2)),
             max(fig_y_2),
             (fig_y_2.index(max(fig_y_2)), round(max(fig_y_2), 2)),
             color='r')
    ax2.grid(True)

    ax3 = fig.add_subplot(144)
    ax3.set(title="label", xlabel='frame', ylabel='diameter stenosis')
    ax3.plot(fig_x_3, fig_y_3)
    ax3.text(fig_y_3.index(max(fig_y_3)),
             max(fig_y_3),
             (fig_y_3.index(max(fig_y_3)), round(max(fig_y_3), 2)),
             color='r')

    ax3.grid(True)
    
    fig.savefig(f'{save_dir}/{title}.png')